In [ ]:
!pip install langchain_community

In [9]:
import pandas as pd
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

In [10]:
!pip install faiss-cpu

In [ ]:
!pip install pypdf

In [11]:
import os
from getpass import getpass
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from huggingface_hub import InferenceClient
from langchain.document_loaders import PyPDFLoader  # Import PDF loader

Essai 1 debut

In [12]:
# Set Hugging Face API token
HUGGINGFACEHUB_API_TOKEN = "API"   # Replace with your actual API key
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [13]:
# Load CSV data
from langchain.document_loaders import CSVLoader # Import the CSVLoader
csv_loader = CSVLoader(file_path='mission_schoolab.csv', encoding='utf-8')  # Replace 'your_csv_file.csv' with your CSV file path
csv_documents = csv_loader.load()

# Combine PDF and CSV documents
#all_documents = documents + csv_documents

In [ ]:

# Initialize the InferenceClient
client = InferenceClient(token=HUGGINGFACEHUB_API_TOKEN)

# Function to load and concatenate text from multiple PDFs
def load_pdfs(pdf_paths):
    documents = []
    for pdf_path in pdf_paths:
        print(f"Chargement du fichier : {pdf_path}")
        pdf_loader = PyPDFLoader(pdf_path)
        documents.extend(pdf_loader.load())
    return documents

# List of PDF paths (replace with your actual PDF paths)
pdf_paths = [
    "pdf1.pdf",
    "pdf2.pdf",
    "pdf3.pdf",
]

# Load and preprocess the PDF files
documents = load_pdfs(pdf_paths)

# Combine PDF and CSV documents
all_documents = documents + csv_documents

# Split the text into smaller chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(all_documents)

# Initialize embeddings
embeddings = HuggingFaceEmbeddings()

# Create a FAISS vector store from the texts
vectorstore = FAISS.from_documents(texts, embeddings)

# Define the prompt template in French
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Étant donné le contexte suivant :

    {context}

    Répondez à la question ci-dessous en suivant le format suivant :

    **Question :** {question}
    **Réponse :** [Votre réponse ici. Soyez concis et précis.]

    Assurez-vous que la réponse ne dépasse pas 2 phrases.

    Si vous n'avez pas de réponse, dites "Aucune réponse trouvée."
    """,
)

# Create the LLMChain
chain = LLMChain(llm=HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1', model_kwargs={'temperature': 0.6}), prompt=prompt)

def semantic_search(query, vectorstore, k=3):
    # Perform a semantic search to find the most relevant documents
    docs = vectorstore.similarity_search(query, k=k)
    return docs

def ask_question(question, vectorstore, chain):
    # Perform semantic search to find relevant context
    context_docs = semantic_search(question, vectorstore)
    context = " ".join([doc.page_content for doc in context_docs])

    # Invoke the chain with the context and question
    result = chain.invoke({"context": context, "question": question})

    # Print the result
    print(result['text'])

# Boucle d'interaction avec l'utilisateur
print("Bienvenue ! Posez-moi des questions sur les documents. Tapez 'exit' pour quitter.")
while True:
    # Demander à l'utilisateur de poser une question
    question = input("\nVous : ")

    # Quitter si l'utilisateur tape 'exit'
    if question.lower() == "exit":
        print("Chatbot : Au revoir !")
        break

    # Poser la question au chatbot
    print("\nChatbot :")
    ask_question(question, vectorstore, chain)

Chargement du fichier : pdf1.pdf
Chargement du fichier : pdf2.pdf
Chargement du fichier : pdf3.pdf


<ipython-input-14-98b820c4674a>:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
<ipython-input-14-98b820c4674a>:31: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hugg

Bienvenue ! Posez-moi des questions sur les documents. Tapez 'exit' pour quitter.

Vous : quelle est la part des missions à impact natif en 2024?

Chatbot :


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



    Étant donné le contexte suivant :

    Retours sur 20 
projets pour 20201.a Sommaire
1. Restitution des observations p.3
a. Retours sur 20 projets pour 2020 p.4
b. Vision sur le nouveau programme p.7
2. Conception programme p.11
a. Convictions nouveau programme p.12
b. Déroulé détaillé p.15
c. Parcours des parties prenantes clésp.31
2 Activer le call to action
Ajoutez le bouton cliquable dans votre profil pour diriger les visiteurs vers votre
site web, votre agenda...
Bonus : Ajouter un boutonpermanent avec LinkedIn Premium

    Répondez à la question ci-dessous en suivant le format suivant :

    **Question :** quelle est la part des missions à impact natif en 2024?
    **Réponse :** [Votre réponse ici. Soyez concis et précis.]

    Assurez-vous que la réponse ne dépasse pas 2 phrases.

    Si vous n'avez pas de réponse, dites "Aucune réponse trouvée."
    
    **Question :** quelle est la part des missions à impact natif en 2024?
    **Réponse :** Il est prévu que 30% des missio

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



    Étant donné le contexte suivant :

    Retours sur 20 
projets pour 20201.a 20 super-conseils
Booster son profilLinkedIn
LE PERQO OCTOBRE 2024 Sommaire
1. Restitution des observations p.3
a. Retours sur 20 projets pour 2020 p.4
b. Vision sur le nouveau programme p.7
2. Conception programme p.11
a. Convictions nouveau programme p.12
b. Déroulé détaillé p.15
c. Parcours des parties prenantes clésp.31
2

    Répondez à la question ci-dessous en suivant le format suivant :

    **Question :** combien de missions en 2024 touchent l’odd 1
    **Réponse :** [Votre réponse ici. Soyez concis et précis.]

    Assurez-vous que la réponse ne dépasse pas 2 phrases.

    Si vous n'avez pas de réponse, dites "Aucune réponse trouvée."
    
    **Question :** quel est le titre du document ?
    **Réponse :** Le PERQO OCTOBRE 2024 Sommaire

Vous : quelle est la part des missions à impact natif en 2024?


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



Chatbot :

    Étant donné le contexte suivant :

    Retours sur 20 
projets pour 20201.a Sommaire
1. Restitution des observations p.3
a. Retours sur 20 projets pour 2020 p.4
b. Vision sur le nouveau programme p.7
2. Conception programme p.11
a. Convictions nouveau programme p.12
b. Déroulé détaillé p.15
c. Parcours des parties prenantes clésp.31
2 Activer le call to action
Ajoutez le bouton cliquable dans votre profil pour diriger les visiteurs vers votre
site web, votre agenda...
Bonus : Ajouter un boutonpermanent avec LinkedIn Premium

    Répondez à la question ci-dessous en suivant le format suivant :

    **Question :** quelle est la part des missions à impact natif en 2024?
    **Réponse :** [Votre réponse ici. Soyez concis et précis.]

    Assurez-vous que la réponse ne dépasse pas 2 phrases.

    Si vous n'avez pas de réponse, dites "Aucune réponse trouvée."
    
    **Question :** quelle est la part des missions à impact natif en 2024?
    **Réponse :** Il est prévu que 30%

In [ ]:
# function
def extract_answer(result):
  x=[result['text']]
  x1 = x[0]

  # Split the string by line breaks
  split_result = x1.split('\n')

  # Extract the last line
  return split_result[-1][16:]

answer = extract_answer(result)

answer

NameError: name 'result' is not defined

Essai 2 fin